In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os,sys
import matplotlib.pyplot as plt
import numpy as np
import torch
import cv2 as cv
import glob
from PIL import Image
from os import listdir

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models

import random

In [ ]:
%cd /content/drive/MyDrive/DRAC2022/Unet

In [ ]:
!python unet.py

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 512, 512]             640
              ReLU-2         [-1, 64, 512, 512]               0
            Conv2d-3         [-1, 64, 512, 512]          36,928
              ReLU-4         [-1, 64, 512, 512]               0
         MaxPool2d-5         [-1, 64, 256, 256]               0
            Conv2d-6        [-1, 128, 256, 256]          73,856
              ReLU-7        [-1, 128, 256, 256]               0
            Conv2d-8        [-1, 128, 256, 256]         147,584
              ReLU-9        [-1, 128, 256, 256]               0
        MaxPool2d-10        [-1, 128, 128, 128]               0
           Conv2d-11        [-1, 256, 128, 128]         295,168
             ReLU-12        [-1, 256, 128, 128]               0
           Conv2d-13        [-1, 256, 128, 128]         590,080
             ReLU-14        [-1, 256, 1

In [9]:
#### prediction
import torch
import math
import pytorch_unet
from torchvision.utils import save_image
import torch.nn.functional as F
import cv2

fold = 2
run = 1
model_name = 'UNet_fold{}_run{}'.format(fold, run)

device = torch.device('cuda')
dtype = torch.cuda.FloatTensor
model = pytorch_unet.UNet(1).to(device)
model.load_state_dict(torch.load('./models/' + model_name))
model.eval()   # Set model to evaluate mode

UNet(
  (dconv_down1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (dconv_down2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (dconv_down3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (dconv_down4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kerne

In [ ]:
def mkdir(path):
    path = path.strip()
    path = path.rstrip("\\")

    isExists = os.path.exists(path)

    if not isExists:
        os.makedirs(path)
        return True
    else:
        return False

In [73]:
ori_path = '../data/ROSE-2/test/original/'
gt_path = '../data/ROSE-2/test/gt/'
save_dir = 'UnetRoseResults/'
mkdir('UnetRoseResults')
imgs = os.listdir(ori_path)
for img_path in imgs:
    if not img_path.endswith('.png'):
        continue

    simple_transform = transforms.ToTensor()
    
    ori = Image.open(ori_path + img_path).resize((512,512))
    ori = ori.convert("L")
    ori = simple_transform(ori).unsqueeze(0)
    pred = model(ori.to(device))
    pred = torch.sigmoid(pred)
    # pred = F.softmax(pred, dim = 1)
    pred = pred.data.cpu().numpy()
    pred = pred[0,0]
    pred_img = np.array(pred * 255, np.uint8)
    _, img = cv2.threshold(pred_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    pink = (255,0,255)
    img = np.expand_dims(img, 2)
    img = np.repeat(img, 3, 2)
    white_pixels = np.where(
        (img[:, :, 0] == 255) & 
        (img[:, :, 1] == 255) & 
        (img[:, :, 2] == 255)
    )
    img[white_pixels] = pink

    gt = cv2.imread(gt_path + img_path)

    res = np.zeros((512, 512*3, 3))
    res[:,:512,:] = gt
    res[:,512:2*512, :] = cv2.resize(cv2.imread(ori_path + img_path), (512,512))
    res[:, 2*512:, :] = img

    cv2.imwrite(save_dir + img_path, res.astype(int))

In [74]:
ori_path = '../A. Segmentation/1. Original Images/a. Training Set/'
save_dir = 'UnetDracResults/'
mkdir('UnetDracResults')

imgs = os.listdir(ori_path)
for img_path in imgs:
    if not img_path.endswith('.png'):
        continue

    simple_transform = transforms.ToTensor()
    
    ori = Image.open(ori_path + img_path).resize((512,512))
    ori = ori.convert("L")
    ori = simple_transform(ori).unsqueeze(0)
    pred = model(ori.to(device))
    pred = torch.sigmoid(pred)
    # pred = F.softmax(pred, dim = 1)
    pred = pred.data.cpu().numpy()
    pred = pred[0,0]
    pred_img = np.array(pred * 255, np.uint8)
    _, img = cv2.threshold(pred_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    pink = (255,0,255)
    img = np.expand_dims(img, 2)
    img = np.repeat(img, 3, 2)
    white_pixels = np.where(
        (img[:, :, 0] == 255) & 
        (img[:, :, 1] == 255) & 
        (img[:, :, 2] == 255)
    )
    img[white_pixels] = pink


    res = np.zeros((512, 512*2, 3))
    res[:,:512, :] = cv2.resize(cv2.imread(ori_path + img_path), (512,512))
    res[:, 512:, :] = img

    cv2.imwrite(save_dir + img_path, res.astype(int))